# Table of Contents
 <p><div class="lev5 toc-item"><a href="#Objective:" data-toc-modified-id="Objective:-00001"><span class="toc-item-num">0.0.0.0.1&nbsp;&nbsp;</span>Objective:</a></div>

##### Objective:
- finding how many days each article is displayed
- most used words in headlines
- author of articles
- articles divided by topic
- are some articles listed twice on the main page?
- word count of each article
- average word count of author
- relationship between type of article and lenght of time on the main page

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from jupyterthemes import jtplot
from pprint import pprint
import datetime as dt
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import csv
from threading import Thread
import queue

%matplotlib inline
%config IPCompleter.greedy=True

jtplot.style('onedork', ticks=True, context='notebook', grid=True, spines=False, gridlines='-')

In [2]:
# Function for gatherinng urls from the main page of Dagbladet.no: 
bad_urls = [
        'https://www.dagbladet.no/oddstips',
        'https://www.dagbladet.no/annonse/',
        'https://www.dagbladet.no/video/',
        'https://www.dagbladet.no/pluss/utgave/',
]

url = 'https://www.dagbladet.no/'

def db_gather_urls(front_page_url):
    driver.get(front_page_url)
    links = driver.find_elements_by_css_selector('article > a')
    links_list = []
    for link in links:
        part_url = str(link.get_attribute('href'))[0:24]
        full_url = str(link.get_attribute('href'))
        if part_url == 'https://www.dagbladet.no' and not any(substring in full_url for substring in bad_urls):
            links_list.append(link.get_attribute('href'))
    return links_list    

In [3]:
# access each article and gather some basic information
def db_article_statistics(url):
    try:
        driver.get(url)
        title = str(driver.find_element_by_class_name('headline').text)
        date = driver.find_element_by_class_name('published').get_attribute('datetime')[:10]
        published = dt.datetime.strptime(date, '%Y-%m-%d').date()
        if driver.find_element_by_class_name('name').text:
            author = driver.find_element_by_class_name('name').text
        else:
            author = ''
        texts = driver.find_elements_by_css_selector('div.body-copy > p')
        counter = 0
        for text in texts:
            counter = counter + len(text.text.split(sep=' '))
        line = f'{url}, {title}, {published}, {author}, {counter} \n'
        return line
    except:
        pass

In [4]:
from selenium.webdriver.firefox.options import Options
options = Options()
options.add_argument("--headless")
driver = webdriver.Firefox(options=options)

In [5]:
def collect_db_headlines():
    today = str(dt.date.today())
    counter = 0
    filename = 'dagbladet_articles.txt'
    with open(filename, 'a') as csv_file:
        #driver = webdriver.PhantomJS(service_args=['--load-images=no'])
        urls = db_gather_urls('https://www.dagbladet.no/')
        print(f'number of dagbladet urls: {len(urls)}')
        for url in urls:
            counter += 1
            line = f'{today}-{counter},{db_article_statistics(url)}'
            csv_file.write(line)
        driver.quit()

In [6]:
collect_db_headlines()

number of dagbladet urls: 158
